In [30]:
import selenium
from selenium import webdriver
import numpy
import pandas as pd
import time
import requests
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import  Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
# Connecting to the webdriver 
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()        # Maximizing the window

In [22]:
driver.get('https://www.easemytrip.com/')
time.sleep(3)

In [23]:
# Let's make a list of locations to scrap data
loc_lst = ['New Delhi', 'Mumbai', 'Bangalore', 'Chennai','Hyderabad ','Lucknow ','Kolkata ','Jaipur ','Cochin ']

In [24]:
# Creating empty lists to fetch required data

Airline_name=[]      # The name of the airline.
Departure_time = []  # The time when the journey starts from the source.
Arrival_time = []    # Time of arrival at the destination.
Duration=[]          # Total duration taken by the flight to reach the destination from the source.
Stops = []           # Total stops between the source and destination.
Source = []          # The source from which the service begins.
Destination = []     # The destination where the service ends.
Meal=[]              # Availability of meals in the flight.
Price=[]             # The price of the flight ticket.
Location=[]          # The location of the flights 

In [25]:
driver.find_element_by_xpath('//div[@class="fss_flex search_colm"]').click()

In [26]:
# Scraping the data from the mentioned url

# Fetching webelement of source 
for x in loc_lst:
    for y in loc_lst:
        if x!=y:                 
            Source = driver.find_element_by_xpath('/html/body/form/div[9]/div[2]/div/div[4]/div/div/div[1]/div/div[1]/div[1]/div[3]/input[2]')
            time.sleep(2)
            Source.clear() 
            Source.send_keys(x)
            time.sleep(2)
            
             # Fetching webelement of destination
            Destination = driver.find_element_by_xpath('/html/body/form/div[9]/div[2]/div/div[4]/div/div/div[1]/div/div[1]/div[1]/div[5]/input[2]')
            Destination.click()
            Destination.clear()
            time.sleep(2)
            Destination.send_keys(y)
            time.sleep(2)
            
            try:
                srch_btn = driver.find_element_by_xpath('//div[@class="fl-col-5"]')
                
                srch_btn.click()
            except NoSuchElementException:
                pass
            time.sleep(2)
            
            # Scrolling the data 5000 times 
            for _ in range(3):
                time.sleep(1)
                driver.execute_script("window.scrollBy(0,5000)")
                time.sleep(5)
                
             # Fetching name of Airline
            try:
                for i in driver.find_elements_by_xpath('//span[@class="txt-r4 ng-binding"]'):
                    Airline_name.append(i.text)
            except NoSuchElementException:
                pass
             # Fetching departure time of the flights
            try:
                for i in driver.find_elements_by_xpath('//span[@class="txt-r2-n ng-binding"]'):
                    Departure_time.append(i.text)
            except NoSuchElementException:
                pass
             # Fetching arrival time of the flights 
            try:
                for i in driver.find_elements_by_xpath('//span[@class="txt-r2-n ng-binding"]'):
                    Arrival_time.append(i.text)
            except NoSuchElementException:
                pass
            # Fetching Duration of flights journey
            try:
                for i in driver.find_elements_by_xpath('//span[@class="dura_md ng-binding"]'):
                    Duration.append(i.text)
            except NoSuchElementException:
                pass
            # Fetching number of stops the flights have between source and destination
            try:
                for i in driver.find_elements_by_xpath('//span[@class="dura_md2 ng-scope"]'):
                    Stops.append(i.text)
            except NoSuchElementException:
                pass
             # Fetching Locations
            try:
                for i in driver.find_elements_by_xpath('//span[@class="txt-r3-n ng-binding"]'):
                    Location.append(i.text)
            except NoSuchElementException:
                pass
            # Fetching Prices of flight tickets
            try:
                for i in driver.find_elements_by_xpath('//div[@class="col-md-8 col-sm-8 col-xs-9 txt-r6-n exPrc ng-binding"]'):
                    Price.append(i.text)
            except NoSuchElementException:
                pass

In [27]:
Departure_time=Departure_time[0:len(Departure_time):2]
Arrival_time=Arrival_time[1:len(Arrival_time):2]
Source = Location[0:len(Location):2]
Destination = Location[1:len(Location):2]

In [28]:
print(len(Price),len(Airline_name),len(Departure_time),len(Arrival_time),len(Duration),len(Stops),len(Source), len(Destination))

2228 2228 2228 2228 2228 2228 2228 2228


In [31]:
# Creating a dataframe for scraped data
data = list(zip(Airline_name,Departure_time,Arrival_time,Duration,Source,Destination,Stops,Price))
df = pd.DataFrame(data, columns = ["Airline",
                                   "Departure_time",
                                   "Time_of_arrival",
                                   "Duration", 
                                   "Source",
                                   "Destination",
                                   "Number_of_stops",
                                   "Price"])

In [32]:
df

,Airline,Departure_time,Time_of_arrival,Duration,Source,Destination,Number_of_stops,Price
0,SpiceJet,16:25,12:40,20h 15m,Delhi,Mumbai,1-stop,"11,492"
1,Air India,20:40,08:10,11h 30m,Delhi,Mumbai,1-stop,"14,901"
2,Air India,16:10,08:10,16h 00m,Delhi,Mumbai,1-stop,"14,901"
3,Air India,18:45,11:55,17h 10m,Delhi,Mumbai,1-stop,"15,090"
4,Air India,18:30,16:25,21h 55m,Delhi,Mumbai,1-stop,"15,195"
...,...,...,...,...,...,...,...,...
2223,Vistara,19:45,19:50,24h 05m,Cochin,Jaipur,2+-stop,"45,806"
2224,Vistara,19:45,19:50,24h 05m,Cochin,Jaipur,2+-stop,"45,806"
2225,Vistara,19:45,19:50,24h 05m,Cochin,Jaipur,2+-stop,"45,806"
2226,Vistara,19:45,19:50,24h 05m,Cochin,Jaipur,2+-stop,"45,806"


In [33]:
# Saving the data into excel file
df.to_excel("Flight_Prices.xlsx")